In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.optimizers import Adam

In [1]:
train_path = 'D:/WORK/Tomato Disease Prediction/5. Project Executable Files/Dataset/train'
val_path = 'D:/WORK/Tomato Disease Prediction/5. Project Executable Files/Dataset/val'

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Apply ImageDataGenerator functionality to Trainset and Test set
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(256, 256),
    batch_size=80,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=(256, 256),
    batch_size=80,
    class_mode='categorical'
)

Found 10000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [7]:
base_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Adding Dense Layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1000, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# The final model
model = Model(inputs=base_model.input, outputs=predictions)

# Configure the Learning Process
for layers in base_model.layers[:140]:
    layers.trainable=False
for layers in base_model.layers[140:]:
    layers.trainable=True

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics='accuracy')


In [6]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15
)

# Save the Model
model.save('tomato_disease_detector.h5')

Epoch 1/15
125/125 [==============================] - 2803s 22s/step - loss: 0.9972 - accuracy: 0.7046 - val_loss: 0.6202 - val_accuracy: 0.8020
Epoch 2/15
125/125 [==============================] - 280s 2s/step - loss: 0.2472 - accuracy: 0.9249 - val_loss: 0.2086 - val_accuracy: 0.9270
Epoch 3/15
125/125 [==============================] - 281s 2s/step - loss: 0.1346 - accuracy: 0.9581 - val_loss: 0.1439 - val_accuracy: 0.9510
Epoch 4/15
125/125 [==============================] - 280s 2s/step - loss: 0.0971 - accuracy: 0.9691 - val_loss: 0.1590 - val_accuracy: 0.9430
Epoch 5/15
125/125 [==============================] - 281s 2s/step - loss: 0.0710 - accuracy: 0.9772 - val_loss: 0.1339 - val_accuracy: 0.9570
Epoch 6/15
125/125 [==============================] - 280s 2s/step - loss: 0.0550 - accuracy: 0.9828 - val_loss: 0.0997 - val_accuracy: 0.9690
Epoch 7/15
125/125 [==============================] - 282s 2s/step - loss: 0.0492 - accuracy: 0.9852 - val_loss: 0.0921 - val_accuracy: 0.97

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
model.save('/content/drive/MyDrive/Colab Notebooks/tomato_disease_detector.h5')

In [8]:
# Import additional libraries for handling image input
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('tomato_disease_detector.h5')

# Define a function to preprocess the input image and make a prediction
def predict_disease(img_path):
    img = image.load_img(img_path, target_size=(256, 256))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model.predict(img_array)
    class_indices = {v: k for k, v in train_generator.class_indices.items()}
    predicted_class = np.argmax(prediction, axis=1)[0]
    disease_name = class_indices[predicted_class]

    return disease_name

# Test the function
img_path = 'im1.JPG'  # Replace with the path to your test image
disease_name = predict_disease(img_path)
print(f"The disease detected is: {disease_name}")


1/1 [==============================] - 5s 5s/step
The disease detected is: Tomato___Spider_mites Two-spotted_spider_mite
